In [ ]:
import numpy as np

In [ ]:
import scipy as sp

In [ ]:
import math 
import copy

In [ ]:
def optimized_gd(x_0, grad, L, K):
    A = initialize_A(K)
    a_1 = A[1] - A[0]
    print('a_1: ', a_1)
    v_0 = x_0 - (A[1]/(a_1*L))*grad(x_0)
    print('v_0: ', v_0)
    g_0 = a_1*grad(x_0)
    print('g_0: ', g_0)
    x_list = [x_0]
    g_list = [g_0]
    v_list = [v_0]
    y_list = []
    x_cur = x_0
    g_cur = g_0
    v_cur = v_0
    for k in range(1, K):
        y_cur = x_cur - (1/L)*grad(x_cur)
        print('y_cur: ', y_cur)
        x_upd = (A[k]/A[k+1])*y_cur + ((A[k+1] - A[k])/A[k+1])*v_cur - (1/(A[k+1] - A[k]))*g_cur
        print('x_upd: ', x_upd)
        v_upd = v_cur - (1/L)*(A[k+1]/(A[k+1] - A[k]))*grad(x_upd)
        print('v_upd: ', v_upd)
        g_upd = g_cur + (A[k+1] - A[k])*grad(x_upd)
        print('g_upd: ', g_upd)
        
        x_list.append(x_upd)
        y_list.append(y_cur)
        v_list.append(v_upd)
        g_list.append(g_upd)
        
        x_cur = copy.deepcopy(x_upd)
        g_cur = copy.deepcopy(g_upd)
        v_cur = copy.deepcopy(v_upd)
        
    x_last = y_cur - (1/L)*g_cur
    
    return x_last, x_list, y_list, v_list, g_list

In [ ]:
def initialize_A(K):
    A = [0] * (K+1)
    for i in range(K+1):
        if i == 0:
            A[0] = 1
        else:
            A[i] = A[i-1]*(1+0.5*A[i-1]-0.5*np.sqrt(A[i-1]*(4+A[i-1])))
    
    return A[::-1]
    
        
    

In [ ]:
A = initialize_A(100)

In [ ]:
A[-1]/A[0]

In [ ]:
len(A)

In [ ]:
def grad_quad(x):
    A = np.array([[100,0],[0,1]])
    b = np.array([1,1])
    return A@x+b

In [ ]:
np.linalg.solve([[100,0],[0,1]],-np.array([1,1]))

In [ ]:
x_last, x_list, y_list, v_list, g_list = optimized_gd(np.array([1,1]), grad_quad, 100, 100)

In [ ]:
def kim_gd(x_0, grad, L, K):
    theta = init_theta(K)
    x_cur = x_0
    y_cur = x_0
    list_x = [x_0]
    list_y = [x_0]
    for i in range(K):
        y_upd = x_cur - (1/L)*grad(x_cur)
        x_upd = y_upd + (theta[i]-1)*(2*theta[i+1]-1)/(theta[i]*(2*theta[i]-1))*(y_upd - y_cur)\
        + (2*theta[i+1]-1)/(2*theta[i]-1)*(y_upd - x_cur)
        
        list_x.append(x_upd)
        list_y.append(y_upd)
        
        y_cur = y_upd
        x_cur = x_upd
        
    return list_x, list_y
    
    

In [ ]:
def init_theta(K):
    theta = [0] * (K+1)
    for i in range(K+1):
        if i == 0:
            theta[0] = 1
        if i>0 and i<K:
            theta[i] = (1+np.sqrt(1+4*theta[i-1]*theta[i-1]))/2
        if i==K:
            theta[i] = (1+np.sqrt(1+8*theta[i-1]*theta[i-1]))/2
    
    return theta[::-1]

In [ ]:
def init_h(i, theta):
#     i = i+1
    h_i = [0]*(i+1)
    h_i[i] = 1 + (2*theta[i+1]-1)/theta[i]
    h_i[i-1] = ((theta[i] - 1)/theta[i-1])*(h_i[i] - 1)
    for j in range(i-2, -1, -1):
        h_i[j] = ((theta[j+1]-1)/theta[j])*h_i[j+1]
    
    return h_i
        

In [ ]:
init_h(4, init_theta(5))

In [ ]:
x_list, y_list = kim_gd(np.array([2,1]), grad_quad, 100, 10000)

In [ ]:
x_list[-1]

In [ ]:
def nesterov(x_0, grad, L, mu, K):
    x_cur = x_0
    y_cur = x_0
    x_list = [x_0]
    y_list = [x_0]
    for i in range(K):
        x_upd = y_cur - (1/L)*(grad(y_cur))
        y_upd = x_upd + ((np.sqrt(L) - np.sqrt(mu))/(np.sqrt(L) + np.sqrt(mu))) * (x_upd - x_cur)
        
        
        x_list.append(x_upd)
        y_list.append(y_upd)
        
        x_cur = x_upd
        y_cur = y_upd
        
    return x_list, y_list
        

In [ ]:
x_0 = np.array([3,1])
L=100
K = 100000
mu = 1

In [ ]:
def grad_quad(x):
    A = np.array([[L,0],[0,1]])
    b = np.array([2,1])
    return A@x+b

In [ ]:
x_list_nes, y_list_nes = nesterov(x_0, grad_quad, L, mu, K)

In [ ]:
x_star = x_list_nes[-1]

In [ ]:
x_star

In [ ]:
norm_list_nes = []
norm_grad_nes=[]
for x in x_list_nes:
    norm_list_nes.append(np.linalg.norm(x-x_star))
    norm_grad_nes.append(np.linalg.norm(grad_quad(x)))
    

In [ ]:
norm_list_nes[-1]

In [ ]:
x_star

In [ ]:
np.linalg.solve([[L,0],[0,1]],-np.array([2,1]))

In [ ]:
x_list_kim, y_list_kim = kim_gd(np.array([2,1]), grad_quad, L, K)

In [ ]:
norm_list_kim = []
norm_grad_kim=[]
for x in x_list_kim:
    norm_list_kim.append(np.linalg.norm(x-x_star))
    norm_grad_kim.append(np.linalg.norm(grad_quad(x)))

In [ ]:
norm_list_kim[-1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(norm_list_kim)
plt.plot(norm_list_nes)
plt.yscale('log')

In [ ]:
# plt.plot(norm_grad_kim)
plt.plot(norm_grad_nes)
plt.yscale('log')

In [ ]:
norm_list_kim[-1]

In [ ]:
def Dima_method(x_0, y_0, eta_x, eta_y, grad_x, grad_y, alpha, K, K_sub):
    x_list = [x_0]
    y_list = [y_0]
    
    x_0_up = x_0
    y_0_up = y_0
    
    x_up_cur = x_0_up
    y_up_cur = y_0_up
    
    x_cur = x_0
    y_cur = y_0
    
    for i in range(K):
        x_down = alpha*x_cur+(1-alpha)*x_up_cur
        y_down = alpha*y_cur+(1-alpha)*y_up_cur
        
        
        x_cur_sub = y_down
        x_prev_sub = y_down
        y_cur_sub = y_down
        
        for j in range(K_sub):
            y_cur_sub = x_cur_sub + (j/(j+2))*(x_cur_sub - x_prev_sub)
            x_upd_sub = y_cur_sub - (1/(L_y+1/(alpha*eta_y)))*(grad_y(x_down, y_cur_sub)\
                                                               +(1/(alpha*eta_y))*(y_cur_sub-y_down))
        

        
            x_prev_sub = x_cur_sub
            x_cur_sub = x_upd_sub
            
        y_up_upd = x_upd_sub
        
        
        
        
        
#         y_up_upd = solver(x_down, y_down, eta_y, alpha)
        
        x_up_upd = x_down - eta_x*alpha*grad_x(x_down, y_up_upd)
        x_upd = (x_cur + alpha*x_down - eta_x*grad_x(x_down,y_up_upd))/(1+alpha)
        y_upd = (y_cur+alpha*y_up_upd-eta_y*grad_y(x_down,y_up_upd))/(1+alpha)
        
        
        x_list.append(x_upd)
        y_list.append(y_upd)
        
        x_up_cur = x_up_upd
        y_up_cur = y_up_upd
        
        x_cur = x_upd
        y_cur = y_upd
        
    return x_list, y_list
        
        
        
    

In [ ]:
x_0 = np.array([1,2])
y_0 = np.array([2,3])
L = 1000
L_x = 10
L_y = 1000
mu_x = 0.1
mu_y = 0.1
eta_x = 1/np.sqrt(mu_x*L_x)
eta_y = (1/mu_y)*(np.sqrt(mu_x/L_x))
alpha = np.sqrt(mu_x/L_x)
C = 2.
K_sub = int(np.sqrt(2*C)*max(1., np.sqrt(eta_y*alpha*L_y)))
K=10000


In [ ]:
K_sub

In [ ]:
A = np.array([[L_x,0,0,0],[0,mu_x,0,0],[0,0,L_y,0],[0,0,0,mu_y]])
b = np.array([2,1,3,1])

In [ ]:
A

In [ ]:
def grad_quad_4(x):
    A = np.array([[L_x,0,0,0],[0,mu_x,0,0],[0,0,L_y,0],[0,0,0,mu_y]])
    b = np.array([2,1,3,1])
    return A@x+b

In [ ]:
def grad_y(x,y):
    vec = np.array(list(x)+list(y))
    res = A@vec+b
    return res[2:4]

In [ ]:
def grad_x(x,y):
    vec = np.array(list(x)+list(y))
    res = A@vec+b
    return res[0:2]

In [ ]:
A

In [ ]:
x_star = np.linalg.solve(A,-b)

In [ ]:
x_star

In [ ]:
K

In [ ]:
K_sub

In [ ]:
x_list_Dima, y_list_Dima = Dima_method(x_0, y_0, eta_x, eta_y,grad_x, grad_y, alpha, 10000, 20)

In [ ]:
print(x_list_Dima[-1])
print(y_list_Dima[-1])

In [ ]:
len(x_list_Dima)


In [ ]:
len(y_list_Dima)

In [ ]:
norm_list_Dima = []
for i in range(len(x_list_Dima)):
    norm_list_Dima.append(np.linalg.norm(np.array(list(x_list_Dima[i])+list(y_list_Dima[i])) - x_star))
    

In [ ]:
norm_list_Dima_new = []
for i in range(len(x_list_Dima)):
    for k in range(K_sub+1):
        norm_list_Dima_new.append(np.linalg.norm(np.array(list(x_list_Dima[i])+list(y_list_Dima[i])) - x_star))
    

In [ ]:
len(norm_list_Dima_new)

In [ ]:
np.array(list(x_list_Dima[-1])+list(y_list_Dima[-1])) 

In [ ]:
plt.plot(norm_list_Dima, label='BAM')
# plt.plot(norm_list_nes, label = 'NAG')
plt.legend()
plt.grid()
plt.ylabel('$\|(x^k,y^k) - (x^*,y^*)\|^2$')
plt.xlabel('iterations')
plt.yscale('log')
plt.xlim(-100,50000)
# plt.savefig('first_plot.pdf')

In [ ]:
# plt.plot(norm_list_Dima_new, label='BAM_y')
plt.plot(norm_list_Dima, label='BAM_x')
plt.plot(norm_list_nes, label = 'NAG')

# plt.plot(norm_list_ACRCD, label = 'ACRCD')
# plt.plot(norm_list_stich_grad_x, label = 'ACDM_x',marker = '+', markevery = 100)


# plt.plot(list_norm_coup_grad_x, label = 'coup_x')

plt.legend()
plt.grid()
plt.ylabel('$\|(x^k,y^k) - (x^*,y^*)\|^2$')
plt.xlabel('number of gradient calls')
plt.yscale('log')
plt.xlim(-100,3000)
plt.ylim(10**(-6),50)
# plt.savefig('prelim_plot.pdf')

In [ ]:
plt.plot(norm_list_Dima_new, label='BAM_y')
plt.plot(norm_list_nes, label = 'NAG')
# plt.plot(norm_list_stich_grad_y, label = 'ACDM_y',marker = 'd',markevery = 400)
# plt.plot(list_norm_coup_grad_y, label = 'coup_y')
plt.legend()
plt.grid()
plt.ylabel('$\|(x^k,y^k) - (x^*,y^*)\|^2$')
plt.xlabel('number of gradient calls')
plt.yscale('log')
plt.xlim(-100,6000)
plt.ylim(10**(-6),50)

In [ ]:
norm_list_Dima[-1]

In [ ]:
x_0_nes = np.array(list(x_0)+list(y_0)) 

In [ ]:
x_0_nes

In [ ]:
x_0

In [ ]:
y_0

In [ ]:
x_list_nes, y_list_nes = nesterov(x_0_nes, grad_quad_4, L,1, 100000)

In [ ]:
x_list_nes[-1]

In [ ]:
x_star

In [ ]:
norm_list_nes = []
for i in range(len(x_list_nes)):
    norm_list_nes.append(np.linalg.norm(x_list_nes[i] - x_star))

In [ ]:
x_0_nes

In [ ]:
A

In [ ]:
grad_quad_4(x_0_nes)

In [ ]:
norm_list_nes[-1]

In [ ]:
np.linalg.norm(x_list_nes[-1] - x_star)

In [ ]:
def ACRCD(alpha, tau, theta, x_0, y_0, d, K):
    x_list = [x_0]
    y_list = [y_0]
    
    x_cur = x_0
    y_cur = y_0
    
    z_cur_x = x_0
    z_cur_y = y_0
    
    q_cur_x = x_0
    q_cur_y = y_0
    
    for i in range(K):
        x_upd = tau*z_cur_x+(1-tau)*q_cur_x
        y_upd = tau*z_cur_y+(1-tau)*q_cur_y
        
        index_p = np.random.choice([0,1],p=[0.5,0.5])
        if index_p == 0:
            q_upd_x = x_upd - (1/L_x)*grad_x(x_upd, y_upd)
            q_upd_y = q_cur_y
        
        if index_p == 1:
            q_upd_y = y_upd - (1/L_y)*grad_y(x_upd, y_upd)
            q_upd_x = q_cur_x
        
        if index_p == 0:
            z_upd_x = z_cur_x - (1/L_x)*alpha*n*grad_x(x_upd,y_upd)
            z_upd_y = z_cur_y
        
        if index_p == 1:
            z_upd_y = z_cur_y - (1/L_y)*alpha*n*grad_y(x_upd,y_upd)
            z_upd_x = z_cur_x
            
        x_list.append(x_upd)
        y_list.append(y_upd)
        
        x_cur = x_upd
        y_cur = y_upd
        
        z_cur_x = z_upd_x
        z_cur_y = z_upd_y
        
        q_cur_x = q_upd_x
        q_cur_y = q_upd_y
        
        
    return x_list, y_list
        
    
    
    
    

In [ ]:
n=4

In [ ]:
x_init = np.array(list(x_0)+list(y_0))
d = 1/2*(x_init.T@A@x_init+b.T@x_init)

In [ ]:

theta = np.linalg.norm(x_init - x_star)
alpha = (1/n)*np.sqrt(theta/d)

In [ ]:
tau = 1/(alpha*n*n+1)


In [ ]:
x_list_ACRCD, y_list_ACRCD = ACRCD(alpha, tau, theta, x_0, y_0, d, 1000*K)

In [ ]:
x_list_ACRCD[-1]

In [ ]:
y_list_ACRCD[-1]

In [ ]:
x_star

In [ ]:
np.linalg.norm(np.array(list(x_list_ACRCD[-1])+list(y_list_ACRCD[-1])) - x_star)

In [ ]:
norm_list_ACRCD = []
for i in range(len(x_list_ACRCD)):
    norm_list_ACRCD.append(np.linalg.norm(np.array(list(x_list_ACRCD[i])+list(y_list_ACRCD[i])) - x_star))

In [ ]:
1/2*(x_init.T@A@x_init+b.T@x_init)

In [ ]:
b.T@x_init

In [ ]:
theta

In [ ]:
np.random.choice([0,1],p=[0.5,0.5])

In [ ]:
def ACDM(x_0, K, L_x, L_y, beta, grad_x, grad_y, mu):
    v_0 = x_0
    A_0 = 0
    B_0 = 1
    beta = 1./2
    
    B_cur = B_0
    A_cur = A_0
    
    p_x = np.sqrt(L_x)/(np.sqrt(L_x)+np.sqrt(L_y))
    p_y = np.sqrt(L_y)/(np.sqrt(L_x)+np.sqrt(L_y))
    S_b = (np.sqrt(L_x)+np.sqrt(L_y))
    
    x_cur = x_0
    v_cur = v_0
    x_list=[x_0]
    x_list_grad_x = [x_0]
    x_list_grad_y = [x_0]
    my_roots_list = []
    
    for i in range(K):
        coor_index = np.random.choice([0,1], p = [p_x,p_y])
        
        my_roots = np.roots([mu - S_b*S_b, B_cur+A_cur*mu, A_cur*B_cur])
        my_roots_list.append(my_roots)
        a_upd = max(my_roots)
        
        A_upd = A_cur + a_upd
        B_upd = B_cur + mu*a_upd
        
        alpha_cur = a_upd/A_upd
        beta_cur = mu*a_upd/B_upd
        
        y_cur = ((1-alpha_cur)*x_cur + alpha_cur*(1-beta_cur)*v_cur)/(1-alpha_cur*beta_cur)
        
        if coor_index == 0:
            
            x_upd = y_cur - (1/L_x)*grad_x(y_cur)
            v_upd = (1-beta_cur)*v_cur+beta_cur*y_cur-(a_upd/(B_upd*p_x))*grad_x(y_cur)
            x_list_grad_x.append(x_upd)
            
        if coor_index == 1:
            
            x_upd = y_cur - (1/L_y)*grad_y(y_cur)
            v_upd = (1-beta_cur)*v_cur+beta_cur*y_cur-(a_upd/(B_upd*p_y))*grad_y(y_cur)
            x_list_grad_y.append(x_upd)
            
        x_list.append(x_upd)
        
        x_cur = x_upd
        v_cur = v_upd
        A_cur = A_upd
        B_cur = B_upd
        
    return x_list, my_roots_list, x_list_grad_x, x_list_grad_y
            
        
    

In [ ]:
np.roots([2,3,1])

In [ ]:
def grad_y_full(x):
    vec = x
    res = A@vec+b
    true_res = res
    true_res[0:1] = 0
    return true_res

In [ ]:
def grad_x_full(x):
    vec = x
    res = A@vec+b
    true_res = res
    true_res[2:4] = 0
    return true_res

In [ ]:
x_0 = np.array([1,2])
y_0 = np.array([2,3])
L = 1000
L_x = 100
L_y = 1000
mu_x = 0.1
mu_y = 0.1
eta_x = 1/np.sqrt(mu_x*L_x)
eta_y = (1/mu_y)*(np.sqrt(mu_x/L_x))
alpha = np.sqrt(mu_x/L_x)
C = 1.
K_sub = int(np.sqrt(2*C)*max(1., np.sqrt(eta_y*alpha*L_y)))
K=10000



In [ ]:
x_init = np.array([1,2,2,3])

In [ ]:
x_list_stich, roots_stich, x_list_grad_x_stich, x_list_grad_y_stich = ACDM(x_init, 12000, L_x, L_y, 0.5, grad_x_full, grad_y_full)

In [ ]:
x_list_stich[-1]

In [ ]:
x_star

In [ ]:
len(roots_stich)

In [ ]:
norm_list_stich = []
for i in range(len(x_list_stich)):
    norm_list_stich.append(np.linalg.norm(x_list_stich[i] - x_star))

In [ ]:
norm_list_stich[-1]

In [ ]:
norm_list_stich_grad_x = []
for i in range(len(x_list_grad_x_stich)):
    norm_list_stich_grad_x.append(np.linalg.norm(x_list_grad_x_stich[i] - x_star))

In [ ]:
norm_list_stich_grad_y = []
for i in range(len(x_list_grad_y_stich)):
    norm_list_stich_grad_y.append(np.linalg.norm(x_list_grad_y_stich[i] - x_star))

In [ ]:
len(x_list_grad_y_stich)

In [ ]:
max(np.roots([2,2,1]))

In [ ]:
x_list_stich[-1]

In [ ]:
x_star

In [ ]:
A

In [ ]:
def linear_coupling(x_0, grad_x, grad_y, L_x, L_y, mu, K):

    S_alpha = np.sqrt(L_x)+np.sqrt(L_y)
    p_x = np.sqrt(L_x)/S_alpha
    p_y = np.sqrt(L_y)/S_alpha
    tau = 2/(1+np.sqrt((4*S_alpha*S_alpha/mu)+1))
    eta = 1/(tau*S_alpha*S_alpha)
    y_0 = x_0
    z_0 = x_0
    list_y = [y_0]
    y_list_grad_x = [y_0]
    y_list_grad_y = [y_0]
    x_cur = x_0
    y_cur = y_0
    z_cur = z_0
    
    for i in range(K):
        x_upd = tau*z_cur + (1-tau)*y_cur
        index_coor = np.random.choice([0,1],p = [p_x, p_y])
        
        if index_coor == 0:
            
            y_upd = x_upd - (1/L_x)*grad_x(x_upd)
            z_upd = (1/(1 + eta * mu))*(z_cur + eta*mu*x_upd - (eta/p_x)*grad_x(x_upd))
            
            
            y_list_grad_x.append(y_upd)
            
        if index_coor == 1:
            
            y_upd = x_upd - (1/L_y)*grad_y(x_upd)
            z_upd = (1/(1 + eta * mu))*(z_cur + eta*mu*x_upd - (eta/p_y)*grad_y(x_upd))
            
            
            y_list_grad_y.append(y_upd)
            
        list_y.append(y_upd)
        
        x_cur = x_upd
        y_cur = y_upd
        z_cur = z_upd
        
        
    return list_y, y_list_grad_x, y_list_grad_y
            
            
            
            
            

In [ ]:
L_x

In [ ]:
L_y

In [ ]:
list_y_coupling, y_list_grad_x_coup, y_list_grad_y_coup = linear_coupling(x_init, grad_x_full, grad_y_full, L_x, L_y, 100000)

In [ ]:
list_y_coupling[-1]

In [ ]:
x_star

In [ ]:
list_norm_coup = []
for i in range(len(list_y_coupling)):
    list_norm_coup.append(np.linalg.norm(list_y_coupling[i] - x_star))

In [ ]:
list_norm_coup[-1]

In [ ]:
list_norm_coup_grad_x = []
for i in range(len(y_list_grad_x_coup)):
    list_norm_coup_grad_x.append(np.linalg.norm(y_list_grad_x_coup[i] - x_star))

In [ ]:
list_norm_coup_grad_y = []
for i in range(len(y_list_grad_y_coup)):
    list_norm_coup_grad_y.append(np.linalg.norm(y_list_grad_y_coup[i] - x_star))

In [ ]:
len(list_norm_coup_grad_x)

In [ ]:
len(list_norm_coup_grad_y)

In [ ]:
len(list_norm_coup_grad_y)

In [ ]:
len(y_list_grad_x_coup)

In [ ]:
y_list_grad_x_coup

In [ ]:
list_norm_coup_grad_y[-1]

# Large experiment

In [ ]:
L_x = 50
L_y = 500
mu = 0.1
eta_x = 1/np.sqrt(mu_x*L_x)
eta_y = (1/mu_y)*(np.sqrt(mu_x/L_x))
alpha = np.sqrt(mu_x/L_x)
C = 1
K_sub = int(np.sqrt(2*C)*max(1., np.sqrt(eta_y*alpha*L_y)))
K=10000

In [ ]:
K_sub

In [ ]:
d_x = 100
d_y = 10

In [ ]:
import scipy as sp
from scipy import stats


In [ ]:
# diag_x = stats.truncnorm.rvs(mu,L_x, size = d_x)

In [ ]:
diag_x = np.sort(np.random.uniform(low=mu, high=L_x, size=d_x))

In [ ]:
diag_x_new = diag_x[::-1]

In [ ]:
diag_x_new[0] = L_x

In [ ]:
diag_x_new[-1] = mu

In [ ]:
diag_x_new

In [ ]:
diag_y = np.sort(np.random.uniform(low=mu, high=L_y, size=d_y))

In [ ]:
# diag_y = stats.truncnorm.rvs(mu,L_y, size = d_y)

In [ ]:
diag_y_new = diag_y[::-1]

In [ ]:
diag_y_new[0] = L_y
diag_y_new[-1] = mu

In [ ]:
diag_y_new

In [ ]:
diag_full = np.array( list(diag_x_new) + list(diag_y_new) )

In [ ]:
A_full = np.diag(diag_full)

In [ ]:
b = np.random.uniform(low=1, high=10, size=(d_y+d_x))

In [ ]:
def quad_function(x):
    res = 0.5*(x.T@A_full@x) - b@x
    return res

In [ ]:
quad_function(np.ones(d_y+d_x))

In [ ]:
x_star = np.linalg.solve(A_full,b)

In [ ]:
quad_function(x_star)

In [ ]:
def grad_x_new(x,y):
    vec = np.array(list(x)+list(y))
    grad = A_full@vec - b
    return grad[0:d_x]

In [ ]:
def grad_x_new_full(x):
    grad = A_full@x - b
    grad[d_x:(d_y+d_x)] = 0.
    return grad

In [ ]:
def grad_y_new(x,y):
    vec = np.array(list(x)+list(y))
    grad = A_full@vec - b
    return grad[d_x:(d_y+d_x)]

In [ ]:
def grad_y_new_full(x):
    grad = A_full@x - b
    grad[0:d_x] = 0.
    return grad

In [ ]:
def grad_new(x):
    grad = A_full@x - b
    return grad


In [ ]:
x_0 = np.zeros(d_x)
y_0 = np.zeros(d_y)

In [ ]:
x_list_large_Dima, y_list_large_Dima = Dima_method(x_0,y_0,eta_x,eta_y,grad_x_new, grad_y_new, alpha,K,K_sub)

In [ ]:
quad_function(np.array(list(x_list_large_Dima[-1])+list(y_list_large_Dima[-1])))

In [ ]:
func_list_large_Dima_x = []
for i in range(len(x_list_large_Dima)):
    func_list_large_Dima_x.append(quad_function(np.array(list(x_list_large_Dima[i])+list(y_list_large_Dima[i]))) - quad_function(x_star))

In [ ]:
func_list_large_Dima_y = []
for i in range(len(x_list_large_Dima)):
    for j in range(K_sub):
        func_list_large_Dima_y.append(quad_function(np.array(list(x_list_large_Dima[i])+list(y_list_large_Dima[i]))) - quad_function(x_star))

In [ ]:
# plt.plot(func_list_large_Dima_x[0:250],label = 'BAM_x')
plt.plot(func_list_large_Dima_2_x[0:320],label = 'BAM_x',marker = 'd', markevery = 20)
# plt.plot(func_list_large_Dima_y[0:4000],label = 'BAM_y')
# plt.plot(func_list_large_Dima_2_y[0:4000],label = 'BAM_y')
plt.plot(func_list_large_Nes_x[0:320], label = 'NAG',marker = 's', markevery = 20)
plt.plot(func_list_large_coup_x[0:320], label = 'LinCoupling_x',marker = 'v', markevery = 20)
plt.plot(func_list_large_ACDM_x[0:320], label = 'ACDM_x',marker = 'o', markevery = 20)
plt.grid()
plt.xlabel('# of oracle calls',fontsize=18)
plt.ylabel('$f(x^k) - f(x^\star)$',fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.ylim(10**(-6),10**4)
plt.yscale('log')
plt.tight_layout()
plt.savefig('10_100_x_E1_50_500.pdf')

In [ ]:
# plt.plot(func_list_large_Dima_x[0:4000],label = 'BAM_x')
# plt.plot(func_list_large_Dima_2_x[0:2000],label = 'BAM_x',marker = 'd', markevery = 100)
# plt.plot(func_list_large_Dima_y[0:4000],label = 'BAM_y')
plt.plot(func_list_large_Dima_2_y[0:1200],label = 'BAM_y',marker = 'd', markevery = 100)
plt.plot(func_list_large_Nes_x[0:1200], label = 'NAG',marker = 's', markevery = 90)
plt.plot(func_list_large_coup_y[0:1200], label = 'LinCoupling_y',marker = 'v', markevery = 100)
plt.plot(func_list_large_ACDM_y[0:1200], label = 'ACDM_y',marker = 'o', markevery = 90)
plt.grid()
plt.xlabel('# of oracle calls',fontsize=18)
plt.ylabel('$f(x^k) - f(x^\star)$',fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.ylim(10**(-9),10**4)
plt.yscale('log')
plt.tight_layout()
plt.savefig('10_100_y_E1_50_500.pdf')

In [ ]:
x_init = np.array(list(x_0)+list(y_0))

In [ ]:
x_list_large_Nes, y_list_large_Nes = nesterov(x_init, grad_new, L_y,mu,K)

In [ ]:
func_list_large_Nes_x = []
for i in range(len(x_list_large_Nes)):
    func_list_large_Nes_x.append(quad_function(x_list_large_Nes[i]) - quad_function(x_star))

In [ ]:
def Dima_method_2(x_0, y_0, eta_x, eta_y, grad_x, grad_y, alpha, K, K_sub):
    x_list = [x_0]
    y_list = [y_0]
    
    x_0_up = x_0
    y_0_up = y_0
    
    x_up_cur = x_0_up
    y_up_cur = y_0_up
    
    x_cur = x_0
    y_cur = y_0
    
    for i in range(K):
        x_down = alpha*x_cur+(1-alpha)*x_up_cur
        y_down = alpha*y_cur+(1-alpha)*y_up_cur
        
        
        x_cur_sub = y_down
        x_prev_sub = y_down
        y_cur_sub = y_down
        
        for j in range(K_sub//2):
            y_cur_sub = x_cur_sub + (j/(j+2))*(x_cur_sub - x_prev_sub)
            x_upd_sub = y_cur_sub - (1/(L_y+1/(alpha*eta_y)))*(grad_y(x_down, y_cur_sub)\
                                                               +(1/(alpha*eta_y))*(y_cur_sub-y_down))
        

        
            x_prev_sub = x_cur_sub
            x_cur_sub = x_upd_sub
            
        y_up_upd = x_upd_sub
        
        x_cur_kim = y_up_upd
        y_cur_kim = y_up_upd
        theta = init_theta(K_sub//2)
        for k in range(K_sub//2):
            y_upd_kim = x_cur_kim - (1/L_y)*grad_y(x_cur_kim)
            
            x_upd_kim = y_upd_kim + (theta[i]-1)*(2*theta[i+1]-1)/(theta[i]*(2*theta[i]-1))*(y_upd_kim - y_cur_kim)\
            + (2*theta[i+1]-1)/(2*theta[i]-1)*(y_upd_kim - x_cur_kim)
            
            y_cur_kim = y_upd_kim
            x_cur_kim = x_upd_kim
            
        y_up_upd = x_upd_kim
        
            
            
            
        
        
        
        
        
#         y_up_upd = solver(x_down, y_down, eta_y, alpha)
        
        x_up_upd = x_down - eta_x*alpha*grad_x(x_down, y_up_upd)
        x_upd = (x_cur + alpha*x_down - eta_x*grad_x(x_down,y_up_upd))/(1+alpha)
        y_upd = (y_cur+alpha*y_up_upd-eta_y*grad_y(x_down,y_up_upd))/(1+alpha)
        
        
        x_list.append(x_upd)
        y_list.append(y_upd)
        
        x_up_cur = x_up_upd
        y_up_cur = y_up_upd
        
        x_cur = x_upd
        y_cur = y_upd
        
    return x_list, y_list
        
        
        
    

In [ ]:
K_sub 

In [ ]:
x_list_large_Dima_2, y_list_large_Dima_2 = Dima_method(x_0,y_0,eta_x,eta_y,grad_x_new, grad_y_new, alpha,K,K_sub)

In [ ]:
quad_function(np.array(list(x_list_large_Dima_2[-1])+list(y_list_large_Dima_2[-1])))

In [ ]:
x_list_large_Dima_2[-1][0]

In [ ]:
x_list_large_Dima[-1][0]

In [ ]:
func_list_large_Dima_2_x = []
for i in range(len(x_list_large_Dima_2)):
    func_list_large_Dima_2_x.append(quad_function(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i]))) - quad_function(x_star))

In [ ]:
func_list_large_Dima_2_y = []
for i in range(len(x_list_large_Dima_2)):
    for j in range(K_sub):
        func_list_large_Dima_2_y.append(quad_function(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i]))) - quad_function(x_star))

In [ ]:
x_list_large_lin_coup, x_list_large_grad_x_lin_coup, x_list_large_grad_y_lin_coup = linear_coupling(x_init, grad_x_new_full, grad_y_new_full, L_x, L_y, mu, K)

In [ ]:
func_list_large_coup_x = []
for i in range(len(x_list_large_grad_x_lin_coup)):
    func_list_large_coup_x.append(quad_function(x_list_large_grad_x_lin_coup[i]) - quad_function(x_star))

In [ ]:
func_list_large_coup_y = []
for i in range(len(x_list_large_grad_y_lin_coup)):
    func_list_large_coup_y.append(quad_function(x_list_large_grad_y_lin_coup[i]) - quad_function(x_star))

In [ ]:
x_list_large_ACDM, my_roots, x_list_large_grad_x_ACDM, x_list_large_grad_y_ACDM = ACDM(x_init, K, L_x, L_y, 0.5, grad_x_new_full, grad_y_new_full)

In [ ]:
func_list_large_ACDM_x = []
for i in range(len(x_list_large_grad_x_ACDM)):
    func_list_large_ACDM_x.append(quad_function(x_list_large_grad_x_ACDM[i]) - quad_function(x_star))

In [ ]:
func_list_large_ACDM_y = []
for i in range(len(x_list_large_grad_y_ACDM)):
    func_list_large_ACDM_y.append(quad_function(x_list_large_grad_y_ACDM[i]) - quad_function(x_star))

## Distance

In [ ]:
dist_list_large_ACDM_x = []
for i in range(len(x_list_large_grad_x_ACDM)):
    dist_list_large_ACDM_x.append(np.linalg.norm(x_list_large_grad_x_ACDM[i] - x_star)**2)

In [ ]:
dist_list_large_ACDM_y = []
for i in range(len(x_list_large_grad_y_ACDM)):
    dist_list_large_ACDM_y.append(np.linalg.norm(x_list_large_grad_y_ACDM[i] - x_star)**2)

In [ ]:
dist_list_large_coup_x = []
for i in range(len(x_list_large_grad_x_lin_coup)):
    dist_list_large_coup_x.append(np.linalg.norm(x_list_large_grad_x_lin_coup[i] - x_star)**2)

In [ ]:
dist_list_large_coup_y = []
for i in range(len(x_list_large_grad_y_lin_coup)):
    dist_list_large_coup_y.append(np.linalg.norm(x_list_large_grad_y_lin_coup[i] - x_star)**2)

In [ ]:
dist_list_large_Dima_2_x = []
for i in range(len(x_list_large_Dima_2)):
    dist_list_large_Dima_2_x.append(np.linalg.norm(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i])) - x_star))

In [ ]:
dist_list_large_Dima_2_y = []
for i in range(len(x_list_large_Dima_2)):
    for j in range(K_sub):
        dist_list_large_Dima_2_y.append(np.linalg.norm(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i])) - x_star))

In [ ]:
dist_list_large_Nes_x = []
for i in range(len(x_list_large_Nes)):
    dist_list_large_Nes_x.append(np.linalg.norm(x_list_large_Nes[i] - x_star))

## Log-regression 

In [ ]:
import matplotlib
import numpy as np
import psutil
import ray
import seaborn as sns
import copy

import matplotlib.pyplot as plt
import numpy.linalg as la
from scipy.sparse import csc_matrix, csr_matrix
from sklearn.datasets import load_svmlight_file, fetch_rcv1

from optmethods.datasets import get_dataset
from optmethods.first_order import Adgd, Gd, Nesterov, RestNest
from optmethods.loss import LogisticRegression
from optmethods.utils import get_trace, relative_round

In [ ]:
import sklearn.datasets
import urllib.request
# bs = 1
# w8a_url = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/w8a"
a1a_url = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a1a"
data_path = './a1a'
# f = urllib.request.urlretrieve(w8a_url, data_path)
f = urllib.request.urlretrieve(a1a_url, data_path)
A, b = sklearn.datasets.load_svmlight_file(data_path)

n, dim = A.shape  # \# of data and dimension for each data: 49749, 300
print(n, dim)
# if n % num_cpus != 0:
#     A = A[:n - (n % num_cpus)]
#     b = b[:n - (n % num_cpus)]

A = A.toarray()
l1 = 0
loss = LogisticRegression(A, b, l1=l1, l2=0)
# n, dim = A.shape
# if n <= 20000 or dim <= 20000:
#     print('Computing the smoothness constant via SVD, it may take a few minutes...')
L = loss.smoothness
# L_batch = loss.batch_smoothness(bs)
# L_max = loss.max_smoothness
# # l2 = 0
# l2 = 1e-4 * L       # here we choose the regularizer to be 1e-5, in paper 1e-4
# loss.l2 = l2

In [ ]:
b_unique = np.unique(b)
if (b_unique == [1, 2]).all():
    # Transform labels {1, 2} to {0, 1}
    b = b - 1
elif (b_unique == [-1, 1]).all():
    # Transform labels {-1, 1} to {0, 1}
    b = (b+1) / 2
else:
    # replace class labels with 0's and 1's
    b = 1. * (b == b[0])

In [ ]:
# from sklearn.datasets import make_classification

In [ ]:
# np.random.seed(42)
# n = 1000
# d = 110

# A, b = make_classification(n_samples = n, n_features=d, n_redundant=0, n_informative=3, n_clusters_per_class=1)

In [ ]:
b = 2*b-1

In [ ]:
np.unique(b)

In [ ]:
# L_max = 0
# mu = 0.1
# for j in range(n):
#     q = A[j].T @ A[j]
#     L_i = (q**2)/4 + mu
#     if L_max < L_i:
#         L_max = L_i

In [ ]:
# L_max

In [ ]:
d_x = 100
d_y = dim - d_x

In [ ]:
L

In [ ]:
d_y

In [ ]:
mu_x = 0.01
mu_y = 0.002

In [ ]:
dim 

In [ ]:
import copy

In [ ]:
def func(x, i, A, b, mu_x, mu_y): 
    return np.log(1+np.exp(-b[i]*(A[i].T @ x))) + mu_x/2 * x[0:d_x].T @ x[0:d_x]+ mu_y/2 * x[d_x:dim].T@x[d_x:dim]

def nabla_func(x, i, A, b, mu_x, mu_y):
    vec_x = copy.deepcopy(x)
    vec_x[d_x:dim] = 0
    vec_y = copy.deepcopy(x)
    vec_y[0:d_x] = 0
#     print(x)
#     print(vec_x)
#     print(vec_y)
    return mu_x*vec_x+mu_y*vec_y - 1/(1 + np.exp(b[i]*(A[i].T @ x))) *b[i] * A[i]

def Func(x):
    s = 0
    for i in range(n):
        s += func(x, i, A, b, mu_x, mu_y)
    return s/n

def nabla_Func(x):
    s = np.zeros(dim)
    for i in range(n):
        s += nabla_func(x, i, A, b, mu_x, mu_y)
    return s/n

In [ ]:
x = np.ones(dim)

In [ ]:
x[0:d_x].T@x[0:d_x]

In [ ]:
x[d_x:dim].T@x[d_x:dim]

In [ ]:
Func(np.ones(dim))

In [ ]:
def nabla_Func_x(x,y):
    vec = np.array(list(x)+list(y))
    s = np.zeros(dim)
    for i in range(n):
        s += nabla_func(vec, i, A, b, mu_x, mu_y)
    res = s/n
    return res[0:d_x]

In [ ]:
def nabla_Func_y(x,y):
    vec = np.array(list(x)+list(y))
    s = np.zeros(dim)
    for i in range(n):
        s += nabla_func(vec, i, A, b, mu_x, mu_y)
    res = s/n
    return res[d_x:dim]

In [ ]:
def nabla_Func_y_full(x):
    s = np.zeros(dim)
    for i in range(n):
        s += nabla_func(x, i, A, b, mu_x, mu_y)
    res = s/n
    res[0:d_x] = 0
    return mu_x/mu_y*res

In [ ]:
nabla_Func_y_full(np.ones(dim))[d_x:dim]

In [ ]:
def nabla_Func_x_full(x):
    s = np.zeros(dim)
    for i in range(n):
        s += nabla_func(x, i, A, b, mu_x, mu_y)
    res = s/n
    res[d_x:dim] = 0
    return res

In [ ]:
nabla_Func_x_full(np.ones(dim))[0:d_x]

In [ ]:
L_x = L
L_y = L
mu_x = 0.01
mu_y = 0.002
eta_x = 1/np.sqrt(mu_x*L_x)
eta_y = (1/mu_y)*(np.sqrt(mu_x/L_x))
alpha = np.sqrt(mu_x/L_x)
C = 1
K_sub = int(np.sqrt(2*C)*max(1., np.sqrt(eta_y*alpha*L_y)))
K=10000

In [ ]:
K_sub

In [ ]:
x_0 = np.zeros(d_x)
y_0 = np.zeros(dim-d_x)

In [ ]:
x_init = np.array(list(x_0)+list(y_0))

In [ ]:
K=2000

In [ ]:
x_list_large_Nes, y_list_large_Nes = nesterov(x_init, nabla_Func, L_y,mu_y,K)

In [ ]:
# nabla_Func(x_list_large_Nes[2]) 

In [ ]:
dist_list_large_Nes_x = []
for i in range(len(x_list_large_Nes)):
    dist_list_large_Nes_x.append(np.linalg.norm(x_list_large_Nes[i] - x_list_large_Nes[-1])**2)

In [ ]:
func_list_large_Nes_x = []
for i in range(len(x_list_large_Nes)):
    func_list_large_Nes_x.append(Func(x_list_large_Nes[i]) - Func(x_list_large_Nes[-1]))

In [ ]:
plt.plot(func_list_large_Nes_x[0:300])
plt.plot(func_list_large_Dima_2_x[0:300])
# plt.plot(func_list_large_Dima_2_y[0:300])
plt.plot(func_list_large_coup_x[0:300])
plt.plot(func_list_large_ACDM_x[0:300])
plt.yscale('log')

In [ ]:
func_list_large_Nes_x[0]

In [ ]:
a_a = nabla_func(np.ones(dim), 1, A, b, mu_x, mu_y)

In [ ]:
b_b = nabla_func(np.zeros(dim)*2, 1, A, b, mu_x, mu_y)

In [ ]:
a_a

In [ ]:
b_b

In [ ]:
b_b

In [ ]:
mu_x

In [ ]:
L_x = L
L_y = L

eta_x = 1/np.sqrt(mu_x*L_x)
eta_y = (1/mu_y)*(np.sqrt(mu_x/L_x))
alpha = np.sqrt(mu_x/L_x)
C = 1
K_sub = int(np.sqrt(2*C)*max(1., np.sqrt(eta_y*alpha*L_y)))
K=2000

In [ ]:
K_sub

In [ ]:
K_sub = 2

In [ ]:
y_0

In [ ]:
x_list_large_Dima_2, y_list_large_Dima_2 = Dima_method(x_0,y_0,eta_x,eta_y,nabla_Func_x, nabla_Func_y, alpha,K,K_sub)

In [ ]:
func_list_large_Dima_2_x = []
for i in range(len(x_list_large_Dima_2)):
    func_list_large_Dima_2_x.append(Func(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i]))) - Func(x_list_large_Nes[-1]))

In [ ]:
func_list_large_Dima_2_y = []
for i in range(len(x_list_large_Dima_2)):
    for j in range(K_sub):
        func_list_large_Dima_2_y.append(Func(np.array(list(x_list_large_Dima_2[i])+list(y_list_large_Dima_2[i]))) - Func(x_list_large_Nes[-1]))

In [ ]:
func_list_large_Dima_2_y = []

In [ ]:
for i in range(len(x_list_large_Dima_2)):
    func_list_large_Dima_2_y+=([func_list_large_Dima_2_x[i]]*K_sub)

In [ ]:
len(func_list_large_Dima_2_y)

In [ ]:
a+=a+([1]*2)

In [ ]:
L_y

In [ ]:
x_list_large_lin_coup, x_list_large_grad_x_lin_coup, x_list_large_grad_y_lin_coup = linear_coupling(x_init, nabla_Func_x_full, nabla_Func_y_full, L_x, (mu_x/mu_y)*L_y, mu_x, K)

In [ ]:
func_list_large_coup_x = []
for i in range(len(x_list_large_grad_x_lin_coup)):
    func_list_large_coup_x.append(Func(x_list_large_grad_x_lin_coup[i]) - Func(x_list_large_Nes[-1]))

In [ ]:
func_list_large_coup_y = []
for i in range(len(x_list_large_grad_y_lin_coup)):
    func_list_large_coup_y.append(Func(x_list_large_grad_y_lin_coup[i]) - Func(x_list_large_Nes[-1]))

In [ ]:
x_list_large_ACDM, my_roots, x_list_large_grad_x_ACDM, x_list_large_grad_y_ACDM = ACDM(x_init, K, L_x, (mu_x/mu_y)*L_y, 0.5, nabla_Func_x_full, nabla_Func_y_full,mu_x)

In [ ]:
func_list_large_ACDM_x = []
for i in range(len(x_list_large_grad_x_ACDM)):
    func_list_large_ACDM_x.append(Func(x_list_large_grad_x_ACDM[i]) - Func(x_list_large_Nes[-1]))

In [ ]:
func_list_large_ACDM_y = []
for i in range(len(x_list_large_grad_y_ACDM)):
    func_list_large_ACDM_y.append(Func(x_list_large_grad_y_ACDM[i]) - Func(x_list_large_Nes[-1]))

In [ ]:
# plt.plot(func_list_large_Dima_x[0:250],label = 'BAM_x')
plt.plot(func_list_large_Dima_2_x[0:200],label = 'BAM_x',marker = 'd', markevery = 20)
# plt.plot(func_list_large_Dima_y[0:4000],label = 'BAM_y')
# plt.plot(func_list_large_Dima_2_y[0:4000],label = 'BAM_y')
plt.plot(func_list_large_Nes_x[0:200], label = 'NAG',marker = 's', markevery = 20)
plt.plot(func_list_large_coup_x[0:200], label = 'LinCoupling_x',marker = 'v', markevery = 20)
plt.plot(func_list_large_ACDM_x[0:200], label = 'ACDM_x',marker = 'o', markevery = 20)
plt.grid()
plt.xlabel('# of oracle calls',fontsize=18)
plt.ylabel('$f(x^k) - f(x^\star)$',fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.ylim(10**(-14),10)
plt.yscale('log')
plt.tight_layout()
plt.savefig('log_001-00-1-upd_x_mm.pdf')

In [ ]:
# plt.plot(func_list_large_Dima_x[0:4000],label = 'BAM_x')
# plt.plot(func_list_large_Dima_2_x[0:2000],label = 'BAM_x',marker = 'd', markevery = 100)
# plt.plot(func_list_large_Dima_y[0:4000],label = 'BAM_y')
plt.plot(func_list_large_Dima_2_y[0:500],label = 'BAM_y',marker = 'd', markevery = 100)
plt.plot(func_list_large_Nes_x[0:500], label = 'NAG',marker = 's', markevery = 90)
plt.plot(func_list_large_coup_y[0:500], label = 'LinCoupling_y',marker = 'v', markevery = 100)
plt.plot(func_list_large_ACDM_y[0:500], label = 'ACDM_y',marker = 'o', markevery = 90)
plt.grid()
plt.xlabel('# of oracle calls',fontsize=18)
plt.ylabel('$f(x^k) - f(x^\star)$',fontsize=18)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend()
plt.ylim(10**(-14),10)
plt.yscale('log')
plt.tight_layout()
plt.savefig('log_001-00-1-upd_y_mm.pdf')

In [ ]:
L

In [ ]:
dim

In [ ]:
A, b = sklearn.datasets.load_svmlight_file(data_path)

In [ ]:
A.toarray()

In [ ]:
A.shape

In [ ]:
mu_x

In [ ]:
mu_y